# Workshop Microsoft - Spain IA - Sostenibilidad
En este caso de uso usaremos un LLM pequeño para evaluar su impacto energético

In [9]:
%pip install transformers==4.48 accelerate sentencepiece tokenizer codecarbon

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 98.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
Note: you may need to restart the kernel to use updated packages.


Reiniciar el kernel

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import textwrap
from codecarbon import EmissionsTracker
from codecarbon import track_emissions


import torch

## Modelo
Usaremos mocrosoft Phi

In [2]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    #device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.26it/s]


Prueba con el texto de ejemplo de huggingface

In [3]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Device set to use cpu
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


 To solve the linear equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation to isolate the term with the variable (x):

   2x + 3 - 3 = 7 - 3
   2x = 4

2. Now, divide both sides of the equation by the coefficient of x (which is 2):

   2x / 2 = 4 / 2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


creamos funciones para organizar output

In [4]:

def wrap_text(text, width=100):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def generate(input_text, system_prompt="", max_length=1024):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":  input_text},
    ]

    output = pipe(messages, **generation_args)
    text = output[0]['generated_text']

    wrapped_text = wrap_text(text)
    return wrapped_text

In [5]:
print(generate('¿Quién eres?',
         system_prompt="Eres marcoaurelio, hijo de thor",
         ))

' Soy un modelo de lenguaje de IA desarrollado por Microsoft. No soy Marco Aurélio, un renombrado\nabogado brasileño, sino una inteligencia artificial diseñada para ayudar con una amplia gama de\npreguntas y tareas proporcionando información y asistencia.'

# Consumo energético con CodeCarbon

In [7]:
%%time
# Start tracking
tracker = EmissionsTracker(tracking_mode="process") # save_to_api=True,
tracker.start()

generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         )
emmissions = tracker.stop()


[codecarbon INFO @ 12:02:31] [setup] RAM Tracking...
[codecarbon INFO @ 12:02:31] [setup] CPU Tracking...
[codecarbon WARNING @ 12:02:31] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:02:32] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz
[codecarbon INFO @ 12:02:32] [setup] GPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `t

CPU times: user 34.3 s, sys: 51.9 ms, total: 34.3 s
Wall time: 20.1 s


In [8]:
print(emmissions)

4.0572470961205693e-05


Usando with

In [9]:

with EmissionsTracker() as tracker:
  generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         )


[codecarbon INFO @ 12:02:51] [setup] RAM Tracking...
[codecarbon INFO @ 12:02:51] [setup] CPU Tracking...
[codecarbon WARNING @ 12:02:51] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:02:52] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz
[codecarbon INFO @ 12:02:52] [setup] GPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `t

Como decorador

In [12]:
@track_emissions(project_name="workshop-sostenibilidad")
def answer_to_different_messages(messages):
  outputs = []
  system_prompt = "You are a friendly and helpful assistant"
  for message in messages:
    output = generate(message)
    outputs.append(message)
  return outputs


In [13]:
answer_to_different_messages(["hola!","cómo puedo hacerlo?"," gracias!"])

[codecarbon INFO @ 12:03:49] [setup] RAM Tracking...
[codecarbon INFO @ 12:03:49] [setup] CPU Tracking...
[codecarbon WARNING @ 12:03:49] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:03:50] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz
[codecarbon INFO @ 12:03:50] [setup] GPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `t

['hola!', 'cómo puedo hacerlo?', ' gracias!']

# Uso de la API

https://dashboard.codecarbon.io/

In [19]:
# en local
#!codecarbon login

                                                                                                    
 Usage: codecarbon login [OPTIONS]                                                                  
                                                                                                    
╭─ Options ────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯



In [17]:

!codecarbon config

Welcome to CodeCarbon configuration wizard
Existing global config file found :
╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.11/dist-packages/codecarbon/cli/main.py:177 in config                     │
│                                                                                                  │
│   174 │                                                                                          │
│   175 │   if global_path.exists():                                                               │
│   176 │   │   print("Existing global config file found :")                                       │
│ ❱ 177 │   │   show_config(global_path)                                                           │
│   178 │   │                                                                                      │
│   179 │   │   use_config = questionary_prompt(                                                   │
│   180 │   

In [11]:
!codecarbon login

⠸ Please complete authentication in your browser.^C


In [12]:
!codecarbon init

Usage: codecarbon [OPTIONS] COMMAND [ARGS]...
Try 'codecarbon --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────────────────────────╮
│ No such command 'init'.                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯


In [ ]:
!codecarbon monitor


In [13]:

%%time
# Start tracking
tracker = EmissionsTracker(tracking_mode="process", save_to_api=True, )
tracker.start()

generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         )
emmissions = tracker.stop()

[codecarbon INFO @ 21:23:56] [setup] RAM Tracking...
[codecarbon INFO @ 21:23:56] [setup] CPU Tracking...
[codecarbon WARNING @ 21:23:56] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 21:23:57] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 21:23:57] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:23:57] [setup] GPU Tracking...
[codecarbon INFO @ 21:23:57] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:23:57] >>> Tracker's metadata:
[codecarbon INFO @ 21:23:57]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 21:23:57]   Python version: 3.11.11
[codecarbon INFO @ 21:23:57]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 21:23:57]   Available RAM : 12.675 GB
[codecarbon INFO @ 21:23:57]   CPU count: 2
[codeca

<class 'str'>


[codecarbon INFO @ 21:24:03] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.6319141387939453 W
[codecarbon INFO @ 21:24:03] Energy consumed for all CPUs : 0.000066 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:24:03] Energy consumed for all GPUs : 0.000063 kWh. Total GPU Power : 40.90357215002361 W
[codecarbon INFO @ 21:24:03] 0.000130 kWh of electricity used since the beginning.
[codecarbon ERROR @ 21:24:03] ApiClient Error when calling the API on https://api.codecarbon.io/runs with : {"timestamp": "2025-02-18T21:24:03.875871+00:00", "experiment_id": "5b0fa12a-3dd7-45bb-9766-cc326314d9f1", "os": "Linux-6.1.85+-x86_64-with-glibc2.35", "python_version": "3.11.11", "codecarbon_version": "2.8.3", "cpu_count": 2, "cpu_model": "Intel(R) Xeon(R) CPU @ 2.00GHz", "gpu_count": 1, "gpu_model": "1 x Tesla T4", "longitude": 6.6, "latitude": 53.2, "region": null, "provider": null, "ram_total_size": 12.67477035522461, "tracking_mode": "process"}
[codecarbon ERROR @ 21:24:03] ApiClient A

<class 'str'>
CPU times: user 3.34 s, sys: 30.7 ms, total: 3.38 s
Wall time: 7.25 s
